In [2]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

warnings.filterwarnings('ignore')

DIR = "C:/Users/Yipeng/Contest/BDCI2023/code/"
DIR_DATA_RAW = DIR + "data/"
DIR_DATA_PRE = DIR + "data/preprocessed/"

# 1. 单个文件数据

## 1.1 `sku_info.csv`

包含sku_id,item_first_cate_cd,item_second_cate_cd,item_third_cate_cd,brand_code

先暂时不做处理

In [3]:
## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [4]:
data_sku_info = pd.read_csv(DIR_DATA_RAW + "sku_info.csv")
data_sku_info = reduce_mem_usage(data_sku_info)
data_sku_info

Mem. usage decreased to  0.01 Mb (84.7% reduction)


,sku_id,item_first_cate_cd,item_second_cate_cd,item_third_cate_cd,brand_code
0,1,1,1,1,1
1,2,1,1,1,2
2,3,1,1,1,2
3,4,1,1,2,3
4,5,1,1,2,4
...,...,...,...,...,...
995,996,2,3,32,126
996,997,2,4,27,13
997,998,2,4,10,13
998,999,2,4,27,8


In [4]:
data_sku_info.to_pickle(DIR_DATA_PRE + "data_sku_info.pkl")
del data_sku_info

## 1.2 `sku_price_and_status.csv`

包含store_id,sku_id,date,salable_status,stock_status,original_price

对从最小日期到最大日期出现original_price缺失的情况进行向前填充

In [29]:
data_sku_price_and_status = pd.read_csv(DIR_DATA_RAW + "sku_price_and_status.csv")
data_sku_price_and_status['date'] = pd.to_datetime(data_sku_price_and_status['date'])
# 按照store_id, sku_id进行分组，然后补齐从最小日期到最大日期的数据 TODO: 暂时做前向填充
data_sku_price_and_status = data_sku_price_and_status.groupby(["store_id", "sku_id"]).apply(lambda x: x.set_index("date").resample("D").ffill()).drop(["store_id", "sku_id"], axis=1).reset_index()
data_sku_price_and_status['date'] = pd.to_datetime(data_sku_price_and_status['date']).dt.date
data_sku_price_and_status.drop(["salable_status", "stock_status"], axis=1, inplace=True)
data_sku_price_and_status

,store_id,sku_id,date,original_price
0,1,1,2021-08-31,25.48
1,1,1,2021-09-01,25.48
2,1,1,2021-09-02,25.48
3,1,1,2021-09-03,25.48
4,1,1,2021-09-04,25.48
...,...,...,...,...
5131420,12,1000,2023-09-10,27.38
5131421,12,1000,2023-09-11,27.38
5131422,12,1000,2023-09-12,27.38
5131423,12,1000,2023-09-13,27.38


In [30]:
data_sku_price_and_status['price_max'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform('max')
data_sku_price_and_status['price_min'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform('min')
data_sku_price_and_status['price_mean'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform('mean')
data_sku_price_and_status['price_std'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform('std')
data_sku_price_and_status['price_median'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform('median')
data_sku_price_and_status['price_skew'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform('skew')
# data_sku_price_and_status['price_kurt'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform('kurt')
data_sku_price_and_status['price_quantile_25'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.25))
data_sku_price_and_status['price_quantile_75'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.75))
data_sku_price_and_status['price_quantile_50'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.50))
data_sku_price_and_status['price_quantile_05'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.05))
data_sku_price_and_status['price_quantile_95'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.95))
data_sku_price_and_status['price_quantile_10'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.10))
data_sku_price_and_status['price_quantile_90'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.90))
# data_sku_price_and_status['price_quantile_15'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.15))
# data_sku_price_and_status['price_quantile_85'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.85))
# data_sku_price_and_status['price_quantile_20'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.20))
# data_sku_price_and_status['price_quantile_80'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.80))
# data_sku_price_and_status['price_quantile_30'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.30))
# data_sku_price_and_status['price_quantile_70'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.quantile(0.70))

data_sku_price_and_status['price_norm'] = data_sku_price_and_status['original_price'] / data_sku_price_and_status['price_max']
# data_sku_price_and_status['original_price'] = data_sku_price_and_status['original_price'].astype(np.float32)
data_sku_price_and_status['price_nunique'] = data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform('nunique')
data_sku_price_and_status['sku_nunique'] = data_sku_price_and_status.groupby(['store_id', 'original_price'])['sku_id'].transform('nunique')
data_sku_price_and_status['price_momentum'] = data_sku_price_and_status['original_price'] / data_sku_price_and_status.groupby(['store_id', 'sku_id'])['original_price'].transform(lambda x: x.shift(1))

data_sku_price_and_status['year'] = pd.to_datetime(data_sku_price_and_status['date']).dt.year
data_sku_price_and_status['month'] = pd.to_datetime(data_sku_price_and_status['date']).dt.month

data_sku_price_and_status['price_momentum_m'] = data_sku_price_and_status['original_price'] / data_sku_price_and_status.groupby(['store_id', 'sku_id', 'month'])['original_price'].transform('mean')
data_sku_price_and_status['price_momentum_y'] = data_sku_price_and_status['original_price'] / data_sku_price_and_status.groupby(['store_id', 'sku_id', 'year'])['original_price'].transform('mean')

data_sku_price_and_status

,store_id,sku_id,date,original_price,price_max,price_min,price_mean,price_std,price_median,price_skew,...,price_quantile_10,price_quantile_90,price_norm,price_nunique,sku_nunique,price_momentum,year,month,price_momentum_m,price_momentum_y
0,1,1,2021-08-31,25.48,25.48,22.38,25.01396,1.108684,25.48,-1.960662,...,22.38,25.48,1.0,2,34,NaN,2021,8,1.063679,1.0
1,1,1,2021-09-01,25.48,25.48,22.38,25.01396,1.108684,25.48,-1.960662,...,22.38,25.48,1.0,2,34,1.0,2021,9,1.023560,1.0
2,1,1,2021-09-02,25.48,25.48,22.38,25.01396,1.108684,25.48,-1.960662,...,22.38,25.48,1.0,2,34,1.0,2021,9,1.023560,1.0
3,1,1,2021-09-03,25.48,25.48,22.38,25.01396,1.108684,25.48,-1.960662,...,22.38,25.48,1.0,2,34,1.0,2021,9,1.023560,1.0
4,1,1,2021-09-04,25.48,25.48,22.38,25.01396,1.108684,25.48,-1.960662,...,22.38,25.48,1.0,2,34,1.0,2021,9,1.023560,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131420,12,1000,2023-09-10,27.38,27.38,27.38,27.38000,0.000000,27.38,0.000000,...,27.38,27.38,1.0,1,102,1.0,2023,9,1.000000,1.0
5131421,12,1000,2023-09-11,27.38,27.38,27.38,27.38000,0.000000,27.38,0.000000,...,27.38,27.38,1.0,1,102,1.0,2023,9,1.000000,1.0
5131422,12,1000,2023-09-12,27.38,27.38,27.38,27.38000,0.000000,27.38,0.000000,...,27.38,27.38,1.0,1,102,1.0,2023,9,1.000000,1.0
5131423,12,1000,2023-09-13,27.38,27.38,27.38,27.38000,0.000000,27.38,0.000000,...,27.38,27.38,1.0,1,102,1.0,2023,9,1.000000,1.0


In [31]:
data_sku_price_and_status = reduce_mem_usage(data_sku_price_and_status)
data_sku_price_and_status.to_pickle(DIR_DATA_PRE + "data_sku_price_and_status.pkl")
del data_sku_price_and_status

Mem. usage decreased to 259.37 Mb (72.4% reduction)


## 1.3 `sku_prom.csv`

包含store_id,sku_id,date,promotion_id,curr_day,total_days,promotion_type,threshold,discount_off,channel


In [8]:
data_sku_prom = pd.read_csv(DIR_DATA_RAW + "sku_prom.csv")
data_sku_prom['date'] = pd.to_datetime(data_sku_prom['date']).dt.date
data_sku_prom["row_num"] = data_sku_prom.groupby(["store_id", "sku_id", "date", "channel"]).cumcount()
data_sku_prom = data_sku_prom.set_index(["store_id", "sku_id", "date", "channel", "row_num"]).unstack(fill_value=0, level=-1)
data_sku_prom.columns = [f'{col}_{row_num}' for col, row_num in data_sku_prom.columns]
data_sku_prom.reset_index(inplace=True)
data_sku_prom = reduce_mem_usage(data_sku_prom)
data_sku_prom

Mem. usage decreased to 102.63 Mb (72.2% reduction)


,store_id,sku_id,date,channel,promotion_id_0,promotion_id_1,promotion_id_2,curr_day_0,curr_day_1,curr_day_2,...,total_days_2,promotion_type_0,promotion_type_1,promotion_type_2,threshold_0,threshold_1,threshold_2,discount_off_0,discount_off_1,discount_off_2
0,1,1,2022-05-27,1,1,0,0,1,0,0,...,0,1,0,0,1.509766,0.0,0.0,0.160034,0.0,0.0
1,1,1,2022-05-27,2,1,0,0,1,0,0,...,0,1,0,0,1.509766,0.0,0.0,0.160034,0.0,0.0
2,1,1,2022-05-28,1,1,0,0,2,0,0,...,0,1,0,0,1.509766,0.0,0.0,0.160034,0.0,0.0
3,1,1,2022-05-28,2,1,0,0,2,0,0,...,0,1,0,0,1.509766,0.0,0.0,0.160034,0.0,0.0
4,1,1,2022-05-29,1,1,0,0,3,0,0,...,0,1,0,0,1.509766,0.0,0.0,0.160034,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196156,12,999,2023-08-22,2,8465,0,0,5,0,0,...,0,4,0,0,2.000000,0.0,0.0,0.250000,0.0,0.0
2196157,12,999,2023-08-23,1,8465,0,0,6,0,0,...,0,4,0,0,2.000000,0.0,0.0,0.250000,0.0,0.0
2196158,12,999,2023-08-23,2,8465,0,0,6,0,0,...,0,4,0,0,2.000000,0.0,0.0,0.250000,0.0,0.0
2196159,12,999,2023-08-24,1,8465,0,0,7,0,0,...,0,4,0,0,2.000000,0.0,0.0,0.250000,0.0,0.0


In [9]:
data_sku_prom.to_pickle(DIR_DATA_PRE + "data_sku_prom.pkl")
del data_sku_prom

## 1.4 `sku_sales.csv`

包含order_id,store_id,sku_id,order_time,quantity,channel

我们对quantity求和即可

In [38]:
data_sku_sales = pd.read_csv(DIR_DATA_RAW + "sku_sales.csv")
data_sku_sales['order_time'] = pd.to_datetime(data_sku_sales['order_time'])
data_sku_sales['date'] = data_sku_sales['order_time'].dt.date
data_sku_sales = data_sku_sales.drop(["order_id", "order_time"], axis=1).groupby(["store_id", "sku_id", "date", "channel"]).sum().reset_index()
data_sku_sales

,store_id,sku_id,date,channel,quantity
11,1,1,2022-06-17,1,1.0
13,1,1,2022-06-18,1,4.0
14,1,1,2022-06-19,1,1.0
15,1,1,2022-06-24,1,1.0
16,1,1,2022-06-26,1,1.0
...,...,...,...,...,...
4130683,12,1000,2023-08-31,1,1.0
4130680,12,1000,2023-08-27,2,1.0
4130681,12,1000,2023-08-28,2,1.0
4130682,12,1000,2023-08-29,2,2.0


In [39]:
data_sku_sales = reduce_mem_usage(data_sku_sales)
data_sku_sales.to_pickle(DIR_DATA_PRE + "data_sku_sales.pkl")
del data_sku_sales

Mem. usage decreased to 86.67 Mb (54.2% reduction)


## 1.5 `store_weather.csv`

包含store_id,date,weather_type,min_temperature,max_temperature


In [13]:
data_store_weather = pd.read_csv(DIR_DATA_RAW + "store_weather.csv")
data_store_weather['date'] = pd.to_datetime(data_store_weather['date']).dt.date
data_store_weather = reduce_mem_usage(data_store_weather)
data_store_weather

Mem. usage decreased to  0.12 Mb (62.5% reduction)


,store_id,date,weather_type,min_temperature,max_temperature
0,3,2022-01-30,14,0.199951,0.310059
1,3,2022-04-13,1,0.360107,0.600098
2,3,2022-07-17,0,0.669922,0.910156
3,3,2022-08-23,7,0.640137,0.759766
4,3,2022-12-17,1,0.049988,0.219971
...,...,...,...,...,...
8142,12,2023-04-06,0,0.350098,0.580078
8143,12,2022-10-10,0,0.310059,0.580078
8144,12,2023-01-20,0,0.090027,0.239990
8145,12,2023-04-12,0,0.449951,0.649902


In [14]:
data_store_weather.to_pickle(DIR_DATA_PRE + "data_store_weather.pkl")
del data_store_weather

# 2. 合并数据

In [5]:
data_sku_info = pd.read_pickle(DIR_DATA_PRE + "data_sku_info.pkl")
data_sku_price_and_status = pd.read_pickle(DIR_DATA_PRE + "data_sku_price_and_status.pkl")
data_sku_prom = pd.read_pickle(DIR_DATA_PRE + "data_sku_prom.pkl")
data_sku_sales = pd.read_pickle(DIR_DATA_PRE + "data_sku_sales.pkl")
data_store_weather = pd.read_pickle(DIR_DATA_PRE + "data_store_weather.pkl")

In [6]:
data_merged = pd.merge(data_sku_price_and_status, data_sku_info, on=["sku_id"], how="left")
data_merged = pd.merge(data_merged, pd.DataFrame({"channel": [1, 2]}), how="cross")
del data_sku_price_and_status, data_sku_info
data_merged = pd.merge(data_merged, data_store_weather, on=["store_id", "date"], how="left")
del data_store_weather
data_merged = pd.merge(data_merged, data_sku_sales, on=["store_id", "sku_id", "date", "channel"], how="left").fillna(0)
del data_sku_sales
data_merged = pd.merge(data_merged, data_sku_prom, on=["store_id", "sku_id", "date", "channel"], how="left").fillna(0)
del data_sku_prom
data_merged

,store_id,sku_id,date,original_price,price_max,price_min,price_mean,price_std,price_median,price_skew,...,total_days_2,promotion_type_0,promotion_type_1,promotion_type_2,threshold_0,threshold_1,threshold_2,discount_off_0,discount_off_1,discount_off_2
0,1,1,2021-08-31,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,2021-08-31,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,2021-09-01,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,2021-09-01,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,2021-09-02,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10262845,12,1000,2023-09-12,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10262846,12,1000,2023-09-13,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10262847,12,1000,2023-09-13,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10262848,12,1000,2023-09-14,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# 对每个date前1,3,5,7,14,21,30,60,90天的平均销量、方差、变化率
data_merged.sort_values(["store_id", "sku_id", "channel", "date"], inplace=True)
for day in [1,3,5,7,14,21,30,60,90]:
    print("processing day: ", day)
    data_merged[f'sales_{day}_mean'] = data_merged.groupby(['store_id', 'sku_id', 'channel'])['quantity'].transform(lambda x: x.shift(1).rolling(day).mean())
    data_merged[f'sales_{day}_std'] = data_merged.groupby(['store_id', 'sku_id', 'channel'])['quantity'].transform(lambda x: x.shift(1).rolling(day).std())
    data_merged[f'sales_{day}_mean_change'] = data_merged.groupby(['store_id', 'sku_id', 'channel'])['quantity'].transform(lambda x: x.shift(1).rolling(day).mean().pct_change())

data_merged.fillna(0, inplace=True)
data_merged

processing day:  1
processing day:  3
processing day:  5
processing day:  7
processing day:  14
processing day:  21
processing day:  30
processing day:  60
processing day:  90


,store_id,sku_id,date,original_price,price_max,price_min,price_mean,price_std,price_median,price_skew,...,sales_21_mean_change,sales_30_mean,sales_30_std,sales_30_mean_change,sales_60_mean,sales_60_std,sales_60_mean_change,sales_90_mean,sales_90_std,sales_90_mean_change
0,1,1,2021-08-31,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,2021-09-01,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,2021-09-02,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,1,2021-09-03,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,1,2021-09-04,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10262841,12,1000,2023-09-10,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10262843,12,1000,2023-09-11,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10262845,12,1000,2023-09-12,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10262847,12,1000,2023-09-13,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
PREDICT_WINDOW = 14 + 1

for day in [1,3,5,7,14,21,30,60,90]:
    print("processing day: ", day)
    data_merged[f'sales_{day}_pw_mean'] = data_merged.groupby(['store_id', 'sku_id', 'channel'])['quantity'].transform(lambda x: x.shift(PREDICT_WINDOW).rolling(day).mean())
    data_merged[f'sales_{day}_pw_std'] = data_merged.groupby(['store_id', 'sku_id', 'channel'])['quantity'].transform(lambda x: x.shift(PREDICT_WINDOW).rolling(day).std())
    data_merged[f'sales_{day}_pw_mean_change'] = data_merged.groupby(['store_id', 'sku_id', 'channel'])['quantity'].transform(lambda x: x.shift(PREDICT_WINDOW).rolling(day).mean().pct_change())

data_merged.fillna(0, inplace=True)
data_merged

processing day:  1
processing day:  3
processing day:  5
processing day:  7
processing day:  14
processing day:  21
processing day:  30
processing day:  60
processing day:  90


,store_id,sku_id,date,original_price,price_max,price_min,price_mean,price_std,price_median,price_skew,...,sales_21_pw_mean_change,sales_30_pw_mean,sales_30_pw_std,sales_30_pw_mean_change,sales_60_pw_mean,sales_60_pw_std,sales_60_pw_mean_change,sales_90_pw_mean,sales_90_pw_std,sales_90_pw_mean_change
0,1,1,2021-08-31,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,2021-09-01,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,2021-09-02,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,1,2021-09-03,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,1,2021-09-04,25.484375,25.484375,22.375,25.015625,1.108398,25.484375,-1.960938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10262841,12,1000,2023-09-10,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10262843,12,1000,2023-09-11,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10262845,12,1000,2023-09-12,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10262847,12,1000,2023-09-13,27.375000,27.375000,27.375,27.375000,0.000000,27.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# 分类编码
cols = [
    ['store_id'],
    ['sku_id'],
    ['channel'],
    ['store_id', 'sku_id'],
    ['store_id', 'channel'],
    ['sku_id', 'channel'],
    ['store_id', 'sku_id', 'channel'],
    ['item_first_cate_cd'],
    ['item_first_cate_cd', 'item_second_cate_cd'],
    ['item_first_cate_cd', 'item_second_cate_cd', 'item_third_cate_cd'],
    ['brand_code'],
    ['item_first_cate_cd', 'item_second_cate_cd', 'item_third_cate_cd', 'brand_code']
]

for col in cols:
    print("encoding:", col)
    col_name = '_'+'_'.join(col)+'_'
    data_merged['enc'+col_name+'mean'] = data_merged.groupby(col)['original_price'].transform('mean').astype(np.float16)
    data_merged['enc'+col_name+'std'] = data_merged.groupby(col)['original_price'].transform('std').astype(np.float16)


encoding: ['store_id']
encoding: ['sku_id']
encoding: ['channel']
encoding: ['store_id', 'sku_id']
encoding: ['store_id', 'channel']
encoding: ['sku_id', 'channel']
encoding: ['store_id', 'sku_id', 'channel']
encoding: ['item_first_cate_cd']
encoding: ['item_first_cate_cd', 'item_second_cate_cd']
encoding: ['item_first_cate_cd', 'item_second_cate_cd', 'item_third_cate_cd']
encoding: ['brand_code']
encoding: ['item_first_cate_cd', 'item_second_cate_cd', 'item_third_cate_cd', 'brand_code']


In [10]:
data_merged['date'] = pd.to_datetime(data_merged['date'])
data_merged['year'] = data_merged['date'].dt.year
data_merged['month'] = data_merged['date'].dt.month
data_merged['day'] = data_merged['date'].dt.day
data_merged['dayofweek'] = data_merged['date'].dt.dayofweek
data_merged['dayofyear'] = data_merged['date'].dt.dayofyear
data_merged['quarter'] = data_merged['date'].dt.quarter
data_merged['is_month_start'] = data_merged['date'].dt.is_month_start
data_merged['is_month_end'] = data_merged['date'].dt.is_month_end
data_merged['is_quarter_start'] = data_merged['date'].dt.is_quarter_start
data_merged['is_quarter_end'] = data_merged['date'].dt.is_quarter_end
data_merged['is_year_start'] = data_merged['date'].dt.is_year_start
data_merged['is_year_end'] = data_merged['date'].dt.is_year_end
data_merged['days_in_month'] = data_merged['date'].dt.days_in_month
data_merged['is_leap_year'] = data_merged['date'].dt.is_leap_year

In [11]:
data_merged = reduce_mem_usage(data_merged)
data_merged.to_pickle(DIR_DATA_PRE + "data_merged.pkl")

Mem. usage decreased to 3846.45 Mb (43.9% reduction)


In [12]:
print(len(data_merged.columns))
print(list(data_merged.columns))

142
['store_id', 'sku_id', 'date', 'original_price', 'price_max', 'price_min', 'price_mean', 'price_std', 'price_median', 'price_skew', 'price_quantile_25', 'price_quantile_75', 'price_quantile_50', 'price_quantile_05', 'price_quantile_95', 'price_quantile_10', 'price_quantile_90', 'price_norm', 'price_nunique', 'sku_nunique', 'price_momentum', 'year', 'month', 'price_momentum_m', 'price_momentum_y', 'item_first_cate_cd', 'item_second_cate_cd', 'item_third_cate_cd', 'brand_code', 'channel', 'weather_type', 'min_temperature', 'max_temperature', 'quantity', 'promotion_id_0', 'promotion_id_1', 'promotion_id_2', 'curr_day_0', 'curr_day_1', 'curr_day_2', 'total_days_0', 'total_days_1', 'total_days_2', 'promotion_type_0', 'promotion_type_1', 'promotion_type_2', 'threshold_0', 'threshold_1', 'threshold_2', 'discount_off_0', 'discount_off_1', 'discount_off_2', 'sales_1_mean', 'sales_1_std', 'sales_1_mean_change', 'sales_3_mean', 'sales_3_std', 'sales_3_mean_change', 'sales_5_mean', 'sales_5_st

In [13]:
for x in list(data_merged.columns):
    print(x)

store_id
sku_id
date
original_price
price_max
price_min
price_mean
price_std
price_median
price_skew
price_quantile_25
price_quantile_75
price_quantile_50
price_quantile_05
price_quantile_95
price_quantile_10
price_quantile_90
price_norm
price_nunique
sku_nunique
price_momentum
year
month
price_momentum_m
price_momentum_y
item_first_cate_cd
item_second_cate_cd
item_third_cate_cd
brand_code
channel
weather_type
min_temperature
max_temperature
quantity
promotion_id_0
promotion_id_1
promotion_id_2
curr_day_0
curr_day_1
curr_day_2
total_days_0
total_days_1
total_days_2
promotion_type_0
promotion_type_1
promotion_type_2
threshold_0
threshold_1
threshold_2
discount_off_0
discount_off_1
discount_off_2
sales_1_mean
sales_1_std
sales_1_mean_change
sales_3_mean
sales_3_std
sales_3_mean_change
sales_5_mean
sales_5_std
sales_5_mean_change
sales_7_mean
sales_7_std
sales_7_mean_change
sales_14_mean
sales_14_std
sales_14_mean_change
sales_21_mean
sales_21_std
sales_21_mean_change
sales_30_mean
sales_

In [14]:
temp = data_merged[data_merged['date'] >= pd.to_datetime("2023-08-31")]

In [15]:
s = temp.iloc[0]

In [16]:
for x in list(temp.columns):
    print(x, s[x])

store_id 1
sku_id 1
date 2023-08-31 00:00:00
original_price 22.38
price_max 25.48
price_min 22.38
price_mean 25.02
price_std 1.108
price_median 25.48
price_skew -1.961
price_quantile_25 25.48
price_quantile_75 25.48
price_quantile_50 25.48
price_quantile_05 22.38
price_quantile_95 25.48
price_quantile_10 22.38
price_quantile_90 25.48
price_norm 0.8784
price_nunique 2
sku_nunique 197
price_momentum 1.0
year 2023
month 8
price_momentum_m 0.934
price_momentum_y 0.9277
item_first_cate_cd 1
item_second_cate_cd 1
item_third_cate_cd 1
brand_code 1
channel 1
weather_type 0
min_temperature 0.6
max_temperature 0.76
quantity 0.0
promotion_id_0 40.0
promotion_id_1 0.0
promotion_id_2 0.0
curr_day_0 7.0
curr_day_1 0.0
curr_day_2 0.0
total_days_0 14.0
total_days_1 0.0
total_days_2 0.0
promotion_type_0 1.0
promotion_type_1 0.0
promotion_type_2 0.0
threshold_0 2.46
threshold_1 0.0
threshold_2 0.0
discount_off_0 0.2
discount_off_1 0.0
discount_off_2 0.0
sales_1_mean 1.0
sales_1_std 0.0
sales_1_mean_chan